In [ ]:
!pip install -U -r requirements.txt

In [ ]:
import json
import os 
import prestodb 
import pandas 
import sqlalchemy

In [ ]:
PRESTO_SERVER = os.environ.get('PRESTO_SERVER')
PRESTO_USER= os.environ.get('PRESTO_USER')
PRESTO_PORT= os.environ.get('PRESTO_PORT')
PRESTO_CATALOG= os.environ.get('PRESTO_CATALOG')

Let's just make sure we have a connection and can query presto

In [ ]:
engine = sqlalchemy.create_engine("presto://%s@%s:%d/%s" %(PRESTO_USER, PRESTO_SERVER, 80, PRESTO_CATALOG))
pandas.read_sql('SHOW CATALOGS', engine)

In [ ]:
kafka_sql = 'select * from "notebook-test"'

kafka_raw_df = pandas.read_sql(kafka_sql, engine)
kafka_df = kafka_raw_df.join(kafka_raw_df._message.apply(json.loads).apply(pandas.Series))
kafka_df = kafka_df.drop(columns=['_message'])

kafka_df.head()

In [ ]:
cust_sql = 'select * from "customer-domain".public.customer'

cust_df = pandas.read_sql(cust_sql, engine)
cust_df.head()

In [ ]:
fin_sql = 'select * from "finance-domain".public.transactions'

fin_df = pandas.read_sql(fin_sql, engine)
fin_df["amount"] = fin_df["amount"].str.replace("$", "")
fin_df["amount"] = fin_df["amount"].astype(float)
fin_df.head()

Let's explore customer 42's transaction information

In [ ]:
fin_df[fin_df['customerid'] == 42]

In [ ]:
cust_df[cust_df['id'] == 42]

Let's look at customer spend by mktsegment

In [ ]:
df = cust_df.join(fin_df.groupby("customerid")["amount"].sum("amount"), lsuffix="customerid", rsuffix="id").dropna()
df.head()

In [ ]:
df.plot.scatter(x="mktsegment", y="amount")

Maybe our chatbot messages have signal